In [ ]:
# Remove input cells at runtime (nbsphinx)
import IPython.core.display as d
d.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# Performance poster layout

**Author:** Dr. Michele Peresano (CEA-Saclay/IRFU/DAp/LEPCHE), 2021

**Description:**

This notebook produces a nice poster layout about the comparison between pipelines.  
It is just a refurbished version of the DL3-level "IRF and sensitivity" notebook.

Latest performance results cannot be shown on this public documentation and are therefore hosted at [this RedMine page](https://forge.in2p3.fr/projects/benchmarks-reference-analysis/wiki/Protopipe_performance_data) .

**Requirements and steps to reproduce:**



## Imports

In [ ]:
# From the standard library
from pathlib import Path

# From pyirf
import pyirf
from pyirf.binning import bin_center
from pyirf.utils import cone_solid_angle

# From other 3rd-party libraries
from yaml import load, FullLoader
import numpy as np
import astropy.units as u
from astropy.io import fits
from astropy.table import QTable, Table, Column
import uproot

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
%matplotlib inline
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

# From protopipe
from protopipe.pipeline.utils import load_config

## Functions

In [ ]:
def plot_background_rate(input_file, ax, label, color):
    rad_max = QTable.read(input_file, hdu='RAD_MAX')[0]
    bg_rate = QTable.read(input_file, hdu='BACKGROUND')[0]

    reco_bins = np.append(bg_rate['ENERG_LO'], bg_rate['ENERG_HI'][-1])

    # first fov bin, [0, 1] deg
    fov_bin = 0
    rate_bin = bg_rate['BKG'].T[:, fov_bin]

    # interpolate theta cut for given e reco bin
    e_center_bg = 0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI'])
    e_center_theta = 0.5 * (rad_max['ENERG_LO'] + rad_max['ENERG_HI'])
    theta_cut = np.interp(e_center_bg, e_center_theta, rad_max['RAD_MAX'].T[:, 0])

    # undo normalization
    rate_bin *= cone_solid_angle(theta_cut)
    rate_bin *= np.diff(reco_bins)
    ax.errorbar(
        0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI']).to_value(u.TeV)[1:-1],
        rate_bin.to_value(1 / u.s)[1:-1],
        xerr=np.diff(reco_bins).to_value(u.TeV)[1:-1] / 2,
        ls='',
        label=label,
        color=color,
    )

## Input data

In [ ]:
# Parametrized cell for all input settings
analyses_directory = "/Users/michele/Applications/ctasoft/dirac/shared_folder/analyses" # path to 'analyses' folder
output_directory = Path.cwd() # default output directory for plots

# PROTOPIPE
analysis_name = "test_from_docker_2"
input_filename = "performance_protopipe_Prod3b_CTANorth_baseline_full_array_Zd20deg_180deg_Time50.00h.fits.gz"

# PROTOPIPE (ANY PREVIOUS VERSION)
load_protopipe_previous = False # If True, compare with a previous release of protopipe
analysis_name_2 = ""
input_filename_2 = input_filename # it is here assumed they have the same name (i.e. same production)

# MARS performance (available here: https://forge.in2p3.fr/projects/step-by-step-reference-mars-analysis/wiki)
load_CTAMARS = False # Enable to compare the CTAN analysis done with CTAMARS (Release 2019)
CTAMARS_input_directory = None
CTAMARS_input_filename = None
CTAMARS_label = None

# ED performance (available here: https://forge.in2p3.fr/projects/cta_analysis-and-simulations/wiki/Prod3b_based_instrument_response_functions)
load_EventDisplay = False # Enable to compare with EventDisplay
EventDisplay_input_directory = None
EventDisplay_input_filename = None
EventDisplay_label = None

# REQUIREMENTS
requirements_input_directory = '/Volumes/DataCEA_PERESANO/Data/CTA/requirements/'
site = 'North'
obs_time = '50h'

In [ ]:
# try to check if user has set some variables from the YAML file

if load_EventDisplay:
    
    try:
        
        if EventDisplay_input_directory is None:
            EventDisplay_input_directory = Path(input_data_EventDisplay["input_directory"])
        if EventDisplay_input_filename is None:
            EventDisplay_input_filename = Path(input_directory_CTAMARS["parent_directory"]) / Path(input_directory_CTAMARS["DL3"]["infile"])
        if EventDisplay_label is None:
            EventDisplay_label = input_data_EventDisplay["label"]
    
    except (NameError, KeyError, TypeError):
        print("WARNING: Some of EVENTDisplay input data is undefined! Please, check the documentation of protopipe-BENCHMARKS.")

if load_CTAMARS: 

    try:

        if CTAMARS_input_directory is None:
            CTAMARS_input_directory = Path(input_data_CTAMARS["parent_directory"]) / Path(input_data_CTAMARS["DL3"]["input_directory"])
        if CTAMARS_input_filename is None:
            CTAMARS_input_filename = Path(input_data_CTAMARS["parent_directory"]) / Path(input_data_CTAMARS["DL3"]["input_file"])
        if CTAMARS_label is None:
            CTAMARS_label = input_data_CTAMARS["label"]

         

    except (NameError, KeyError, TypeError):
        print("WARNING: CTAMARS and/or EVENTDisplay input paths undefined! Please, check the documentation of protopipe-BENCHMARKS.")

### Protopipe

In [ ]:
# PREVIOUS VERSION (if any)
if load_protopipe_previous:
    
    try:
        config_performance_old = load_config(Path(analyses_directory) / analysis_name_2 / Path("configs/performance.yaml"))
        obs_time_old = f'{config_performance_old["analysis"]["obs_time"]["value"]}{config_performance_old["analysis"]["obs_time"]["unit"]}'
        production_old = input_filename_2.split("protopipe_")[1].split("_Time")[0]
        protopipe_file_old = Path(analyses_directory, analysis_name_2, "data/DL3", input_filename_2)
    except (NameError, KeyError):
        print("WARNING: you required to compare to data from a previous release, but it is not available!")

In [ ]:
#CURRENT VERSION
config_performance = load_config(Path(analyses_directory) / analysis_name / Path("configs/performance.yaml"))
obs_time = f'{config_performance["analysis"]["obs_time"]["value"]}{config_performance["analysis"]["obs_time"]["unit"]}'
production = input_filename.split("protopipe_")[1].split("_Time")[0]
protopipe_file = Path(analyses_directory, analysis_name, "data/DL3", input_filename)
#protopipe_file = Path("/Users/michele/Applications/ctasoft/dirac/shared_folder") / input_filename

### ASWG

In [ ]:
CTAMARS_performance = uproot.open(Path(CTAMARS_input_directory) / CTAMARS_input_filename)
EventDisplay_performance = uproot.open(Path(EventDisplay_input_directory) / EventDisplay_input_filename)

### Requirements

In [ ]:
requirements_input_filenames = {"sens" : f'/{site}-{obs_time}.dat',
                                "AngRes" : f'/{site}-{obs_time}-AngRes.dat',
                                "ERes" : f'/{site}-{obs_time}-ERes.dat'}
requirements = {}

for key in requirements_input_filenames.keys():
    requirements[key] = Table.read(requirements_input_directory + requirements_input_filenames[key], format='ascii')
requirements['sens'].add_column(Column(data=(10**requirements['sens']['col1']), name='ENERGY'))
requirements['sens'].add_column(Column(data=requirements['sens']['col2'], name='SENSITIVITY'))

## Poster plot

In [ ]:
# First we check if a _plots_ folder exists already.  
# If not, we create it.
plots_folder = Path(output_directory) / "plots"
plots_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# color legend
color_protopipe_current = "Red"
color_protopipe_previous = "DarkOrange"
color_CTAMARS = "DarkBlue"
color_EventDisplay = "DarkGreen"
color_requirements = "black"

In [ ]:
fig = plt.figure(figsize = (20, 10), constrained_layout=True)

gs = fig.add_gridspec(3, 3, figure=fig)

# ==========================================================================================================
#
#                                       SENSITIVITY
#
# ==========================================================================================================

ax1 = fig.add_subplot(gs[0:-1, 0:-1])

# [1:-1] removes under/overflow bins
sensitivity_protopipe = QTable.read(protopipe_file, hdu='SENSITIVITY')[1:-1]

unit = u.Unit('erg cm-2 s-1')

# Add requirements
ax1.plot(requirements['sens']['ENERGY'], 
         requirements['sens']['SENSITIVITY'], 
         color=color_requirements, 
         ls='--', 
         lw=2, 
         label='Requirements'
)

# protopipe
e = sensitivity_protopipe['reco_energy_center']
w = (sensitivity_protopipe['reco_energy_high'] - sensitivity_protopipe['reco_energy_low'])
s_p = (e**2 * sensitivity_protopipe['flux_sensitivity'])
ax1.errorbar(
    e.to_value(u.TeV),
    s_p.to_value(unit),
    xerr=w.to_value(u.TeV) / 2,
    ls='',
    label='protopipe',
    color=color_protopipe_current
)

if load_protopipe_previous:
    
    unit = u.Unit('erg cm-2 s-1')
    sensitivity_protopipe_old = QTable.read(protopipe_file_old, hdu='SENSITIVITY')[1:-1]
    e_old = sensitivity_protopipe_old['reco_energy_center']
    w_old = (sensitivity_protopipe_old['reco_energy_high'] - sensitivity_protopipe_old['reco_energy_low'])
    s_p_old = (e_old**2 * sensitivity_protopipe_old['flux_sensitivity'])
    ax1.errorbar(
        e_old.to_value(u.TeV),
        s_p_old.to_value(unit),
        xerr=w_old.to_value(u.TeV) / 2,
        ls='',
        label='protopipe',
        color=color_protopipe_previous
    )

# ED
if load_EventDisplay:
    s_ED, edges = EventDisplay_performance["DiffSens"].to_numpy()
    yerr = EventDisplay_performance["DiffSens"].errors()
    bins = 10**edges
    x = bin_center(bins)
    width = np.diff(bins)
    ax1.errorbar(
        x,
        s_ED, 
        xerr=width/2,
        yerr=yerr,
        label=EventDisplay_label,
        ls='',
        color=color_EventDisplay
    )

# MARS
if load_CTAMARS:
    s_MARS, edges = CTAMARS_performance["DiffSens"].to_numpy()
    yerr = CTAMARS_performance["DiffSens"].errors()
    bins = 10**edges
    x = bin_center(bins)
    width = np.diff(bins)
    ax1.errorbar(
        x,
        s_MARS, 
        xerr=width/2,
        yerr=yerr,
        label=CTAMARS_label,
        ls='',
        color=color_CTAMARS
    )

# Style settings
ax1.set_xscale("log")
ax1.set_yscale("log")
ax1.set_ylabel(rf"$(E^2 \cdot \mathrm{{Flux Sensitivity}}) /$ ({unit.to_string('latex')})")
ax1.set_ylim(5.e-14, 3.e-10)
ax1.grid(which="both")
ax1.legend()

# ==========================================================================================================
#
#                                       SENSITIVITY RATIO
#
# ==========================================================================================================



ax2 = fig.add_subplot(gs[2, 0])

if load_protopipe_previous:
    
    ax2.errorbar(
        e.to_value(u.TeV), 
        s_p.to_value(unit) / s_p_old.to_value(unit),
        xerr=w.to_value(u.TeV)/2,
        ls='',
        label = "",
        color=color_protopipe_previous
    )

if load_EventDisplay:

    ax2.errorbar(
        e.to_value(u.TeV), 
        s_p.to_value(unit) / s_ED,
        xerr=w.to_value(u.TeV)/2,
        ls='',
        label = "",
        color=color_EventDisplay
    )

if load_CTAMARS:

    ax2.errorbar(
        e.to_value(u.TeV), 
        s_p.to_value(unit) / s_MARS,
        xerr=w.to_value(u.TeV)/2,
        ls='',
        label = "",
        color=color_CTAMARS
    )
ax2.axhline(1, color = color_protopipe_current)

ax2.set_xscale('log')
ax2.set_xlabel("Reconstructed energy [TeV]")
ax2.set_ylabel('Sensitivity ratio')
ax2.grid()

ax2.set_ylim(-1.0, 3.0)

# ==========================================================================================================
#
#                                       EFFECTIVE COLLECTION AREA
#
# ==========================================================================================================

ax3 = fig.add_subplot(gs[0, 2])

# protopipe
# uncomment the other strings to see effective areas
# for the different cut levels. Left out here for better
# visibility of the final effective areas.
suffix =''
#'_NO_CUTS'
#'_ONLY_GH'
#'_ONLY_THETA'

area = QTable.read(protopipe_file, hdu='EFFECTIVE AREA' + suffix)[0]
ax3.errorbar(
    0.5 * (area['ENERG_LO'] + area['ENERG_HI']).to_value(u.TeV),#[1:-1],
    area['EFFAREA'].to_value(u.m**2).T[:,0],#[1:-1, 0],
    xerr=0.5 * (area['ENERG_LO'] - area['ENERG_HI']).to_value(u.TeV),#[1:-1],
    ls='',
    label='protopipe ' + suffix,
    color=color_protopipe_current
)

if load_protopipe_previous:
    
    area_old = QTable.read(protopipe_file_old, hdu='EFFECTIVE AREA' + suffix)[0]
    ax3.errorbar(
        0.5 * (area_old['ENERG_LO'] + area_old['ENERG_HI']).to_value(u.TeV),#[1:-1],
        area_old['EFFAREA'].to_value(u.m**2).T[:,0],#[1:-1, 0],
        xerr=0.5 * (area_old['ENERG_LO'] - area_old['ENERG_HI']).to_value(u.TeV),#[1:-1],
        ls='',
        label='protopipe previous release' + suffix,
        color=color_protopipe_previous
    )

    
# ED
if load_EventDisplay:
    y, edges = EventDisplay_performance["EffectiveAreaEtrue"].to_numpy()
    yerr = EventDisplay_performance["EffectiveAreaEtrue"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    ax3.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=EventDisplay_label,
                 color=color_EventDisplay
                )

# MARS
if load_CTAMARS:
    y, edges = CTAMARS_performance["EffectiveAreaEtrue"].to_numpy()
    yerr = CTAMARS_performance["EffectiveAreaEtrue"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    ax3.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=CTAMARS_label,
                 color=color_CTAMARS
                )

# Style settings
ax3.set_xscale("log")
ax3.set_yscale("log")
ax3.set_ylim(5e1, 2.e6)
ax3.set_xlabel("True energy [TeV]")
ax3.set_ylabel("Effective area [m²]")
ax3.grid(which="both")

# ==========================================================================================================
#
#                                       ANGULAR RESOLUTION
#
# ==========================================================================================================


ax4 = fig.add_subplot(gs[2, 1])

# protopipe
ang_res = QTable.read(protopipe_file, hdu='ANGULAR_RESOLUTION')#[1:-1]

ax4.errorbar(
    0.5 * (ang_res['reco_energy_low'] + ang_res['reco_energy_high']).to_value(u.TeV),
    ang_res['angular_resolution'].to_value(u.deg),
    xerr=0.5 * (ang_res['reco_energy_high'] - ang_res['reco_energy_low']).to_value(u.TeV),
    ls='',
    label='protopipe',
    color=color_protopipe_current
)

# protopipe previous release
if load_protopipe_previous:
    
    ang_res_old = QTable.read(protopipe_file_old, hdu='ANGULAR_RESOLUTION')#[1:-1]

    ax4.errorbar(
        0.5 * (ang_res_old['reco_energy_low'] + ang_res_old['reco_energy_high']).to_value(u.TeV),
        ang_res_old['angular_resolution'].to_value(u.deg),
        xerr=0.5 * (ang_res_old['reco_energy_high'] - ang_res_old['reco_energy_low']).to_value(u.TeV),
        ls='',
        label='protopipe previous release',
        color=color_protopipe_previous
    )


# ED
if load_EventDisplay:
    y, edges = EventDisplay_performance["AngRes"].to_numpy()
    yerr = EventDisplay_performance["AngRes"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    ax4.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=EventDisplay_label,
                 color=color_EventDisplay)

# MARS
if load_CTAMARS:
    y, edges = CTAMARS_performance["AngRes"].to_numpy()
    yerr = CTAMARS_performance["AngRes"].errors()
    x = bin_center(10**edges)
    xerr = 0.5 * np.diff(10**edges)
    ax4.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=CTAMARS_label,
                 color=color_CTAMARS)

# Requirements
ax4.plot(10**requirements['AngRes']['col1'], 
         requirements['AngRes']['col2'], 
         color=color_requirements, 
         ls='--', 
         lw=2, 
         label='Requirements'
)

# Style settings
ax4.set_xscale("log")
#ax4.set_yscale("log")
ax4.set_ylim(2.e-2, 6.e-1)
ax4.set_xlabel("Reconstructed energy [TeV]")
ax4.set_ylabel("Angular resolution [deg]")
ax4.grid(which="both")

None # to remove clutter by mpl objects

# ==========================================================================================================
#
#                                       ENERGY RESOLUTION
#
# ==========================================================================================================


ax5 = fig.add_subplot(gs[2, 2])

# protopipe
bias_resolution = QTable.read(protopipe_file, hdu='ENERGY_BIAS_RESOLUTION')#[1:-1]
ax5.errorbar(
    0.5 * (bias_resolution['reco_energy_low'] + bias_resolution['reco_energy_high']).to_value(u.TeV),
    bias_resolution['resolution'],
    xerr=0.5 * (bias_resolution['reco_energy_high'] - bias_resolution['reco_energy_low']).to_value(u.TeV),
    ls='',
    label='protopipe',
    color=color_protopipe_current
)

# protopipe previous release
if load_protopipe_previous:
    bias_resolution_old = QTable.read(protopipe_file_old, hdu='ENERGY_BIAS_RESOLUTION')#[1:-1]
    ax5.errorbar(
        0.5 * (bias_resolution_old['reco_energy_low'] + bias_resolution_old['reco_energy_high']).to_value(u.TeV),
        bias_resolution['resolution'],
        xerr=0.5 * (bias_resolution_old['reco_energy_high'] - bias_resolution_old['reco_energy_low']).to_value(u.TeV),
        ls='',
        label='protopipe previous release',
        color=color_protopipe_previous
    )

# ED
if load_EventDisplay:
    y, edges = EventDisplay_performance["ERes"].to_numpy()
    yerr = EventDisplay_performance["ERes"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    ax5.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=EventDisplay_label,
                 color=color_EventDisplay
                )

# MARS
if load_CTAMARS:
    y, edges = CTAMARS_performance["ERes"].to_numpy()
    yerr = CTAMARS_performance["ERes"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    ax5.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=CTAMARS_label,
                 color=color_CTAMARS
                )

# Requirements
ax5.plot(10**requirements['ERes']['col1'], 
         requirements['ERes']['col2'], 
         color=color_requirements, 
         ls='--', 
         lw=2, 
         label='Requirements'
)

# Style settings
ax5.set_xlabel("Reconstructed energy [TeV]")
ax5.set_ylabel("Energy resolution")
ax5.grid(which="both")
ax5.set_xscale('log')
ax5.set_ylim(-0.095, 0.45)

None # to remove clutter by mpl objects

# ==========================================================================================================
#
#                                       BACKGROUND RATE
#
# ==========================================================================================================



ax6 = fig.add_subplot(gs[1, 2])

# protopipe
rad_max = QTable.read(protopipe_file, hdu='RAD_MAX')[0]
bg_rate = QTable.read(protopipe_file, hdu='BACKGROUND')[0]

reco_bins = np.append(bg_rate['ENERG_LO'], bg_rate['ENERG_HI'][-1])

# first fov bin, [0, 1] deg
fov_bin = 0
rate_bin = bg_rate['BKG'].T[:, fov_bin]

# interpolate theta cut for given e reco bin
e_center_bg = 0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI'])
e_center_theta = 0.5 * (rad_max['ENERG_LO'] + rad_max['ENERG_HI'])
theta_cut = np.interp(e_center_bg, e_center_theta, rad_max['RAD_MAX'].T[:, 0])

# undo normalization
rate_bin *= cone_solid_angle(theta_cut)
rate_bin *= np.diff(reco_bins)
ax6.errorbar(
    0.5 * (bg_rate['ENERG_LO'] + bg_rate['ENERG_HI']).to_value(u.TeV),#[1:-1],
    rate_bin.to_value(1 / u.s),#[1:-1],
    xerr=np.diff(reco_bins).to_value(u.TeV)/2,#[1:-1] / 2,
    ls='',
    label='protopipe',
    color=color_protopipe_current
)

# protopipe previous release
if load_protopipe_previous:
    
    plot_background_rate(protopipe_file_old, ax6, 'protopipe previous release', color_protopipe_previous)
    

# ED
if load_EventDisplay:
    y, edges = EventDisplay_performance["BGRate"].to_numpy()
    yerr = EventDisplay_performance["BGRate"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    ax6.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=EventDisplay_label,
                 color=color_EventDisplay)


# MARS
if load_CTAMARS:
    y, edges = CTAMARS_performance["BGRate"].to_numpy()
    yerr = CTAMARS_performance["BGRate"].errors()
    x = bin_center(10**edges)
    xerr = np.diff(10**edges) / 2
    ax6.errorbar(x, 
                 y, 
                 xerr=xerr, 
                 yerr=yerr, 
                 ls='', 
                 label=CTAMARS_label,
                 color=color_CTAMARS)


# Style settings
ax6.set_xscale("log")
ax6.set_xlabel("Reconstructed energy [TeV]")
ax6.set_ylabel("Background rate [s⁻¹ TeV⁻¹] ")
ax6.grid(which="both")
ax6.set_yscale('log')
ax6.set_ylim(1.e-10, 1.0)

fig.suptitle(f'{production} - {obs_time}', fontsize=25)
fig.savefig(plots_folder / f"protopipe_{production}_{obs_time}.png")
None # to remove clutter by mpl objects